# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/9822/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

['/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-01.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-02.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-03.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-04.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-05.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-06.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-08.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-09.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-10.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-11.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-12.csv',
 '/home/runner/work/dask-exa

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,951,Kevin,0.9953243568570871,-0.8881208516712078
2000-01-01 00:00:01,1011,Victor,-0.358552611123355,0.2210651092784235
2000-01-01 00:00:02,1033,Alice,-0.209165680258403,0.5720461968083743
2000-01-01 00:00:03,1054,Yvonne,-0.9779528650887441,0.9669696123008236
2000-01-01 00:00:04,1002,Laura,-0.5135838233022125,0.8978512434129124
2000-01-01 00:00:05,988,Hannah,-0.7056261487791728,0.14878597295785778
2000-01-01 00:00:06,974,Sarah,0.5687208830404793,0.8836602009155916
2000-01-01 00:00:07,1015,Hannah,0.11915598946530048,0.8233595271986769
2000-01-01 00:00:08,1004,Michael,0.01090067925240068,-0.4919575664432525


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,994,Tim,0.3657926073459936,-0.6034519859273577
2000-01-30 00:00:01,996,Wendy,0.24263672210997078,0.9363314554432152
2000-01-30 00:00:02,949,Dan,-0.6132590576671486,0.1393221818036412
2000-01-30 00:00:03,992,Xavier,-0.8901064840790973,-0.559256678005376
2000-01-30 00:00:04,1024,Kevin,0.729103150227844,-0.3288414648641851
2000-01-30 00:00:05,986,Jerry,0.3191966308561689,0.9204960655119467
2000-01-30 00:00:06,1033,Ingrid,-0.8091094676850323,-0.4050664939185371
2000-01-30 00:00:07,978,Laura,-0.19211527865274558,0.1315608375458901
2000-01-30 00:00:08,1009,Laura,0.5251693438398084,-0.9987232087731601


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,951,Kevin,0.995324,-0.888121
1,2000-01-01 00:00:01,1011,Victor,-0.358553,0.221065
2,2000-01-01 00:00:02,1033,Alice,-0.209166,0.572046
3,2000-01-01 00:00:03,1054,Yvonne,-0.977953,0.966970
4,2000-01-01 00:00:04,1002,Laura,-0.513584,0.897851


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,951,Kevin,0.995324,-0.888121
1,2000-01-01 00:00:01,1011,Victor,-0.358553,0.221065
2,2000-01-01 00:00:02,1033,Alice,-0.209166,0.572046
3,2000-01-01 00:00:03,1054,Yvonne,-0.977953,0.966970
4,2000-01-01 00:00:04,1002,Laura,-0.513584,0.897851


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.5 s, sys: 479 ms, total: 3.98 s
Wall time: 2.76 s


name
Alice      -0.000647
Bob         0.005896
Charlie    -0.004283
Dan        -0.000023
Edith      -0.001054
Frank       0.001055
George      0.002729
Hannah      0.000738
Ingrid     -0.000145
Jerry      -0.000259
Kevin      -0.003872
Laura      -0.000815
Michael     0.002716
Norbert    -0.003374
Oliver      0.000688
Patricia   -0.000154
Quinn       0.001189
Ray        -0.000154
Sarah       0.000727
Tim        -0.000279
Ursula      0.000606
Victor     -0.001836
Wendy       0.002251
Xavier     -0.001092
Yvonne      0.002094
Zelda       0.003271
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.08 s, sys: 132 ms, total: 1.22 s
Wall time: 911 ms


name
Alice      -0.000647
Bob         0.005896
Charlie    -0.004283
Dan        -0.000023
Edith      -0.001054
Frank       0.001055
George      0.002729
Hannah      0.000738
Ingrid     -0.000145
Jerry      -0.000259
Kevin      -0.003872
Laura      -0.000815
Michael     0.002716
Norbert    -0.003374
Oliver      0.000688
Patricia   -0.000154
Quinn       0.001189
Ray        -0.000154
Sarah       0.000727
Tim        -0.000279
Ursula      0.000606
Victor     -0.001836
Wendy       0.002251
Xavier     -0.001092
Yvonne      0.002094
Zelda       0.003271
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.05 s, sys: 86 ms, total: 1.14 s
Wall time: 911 ms


name
Alice      -0.000647
Bob         0.005896
Charlie    -0.004283
Dan        -0.000023
Edith      -0.001054
Frank       0.001055
George      0.002729
Hannah      0.000738
Ingrid     -0.000145
Jerry      -0.000259
Kevin      -0.003872
Laura      -0.000815
Michael     0.002716
Norbert    -0.003374
Oliver      0.000688
Patricia   -0.000154
Quinn       0.001189
Ray        -0.000154
Sarah       0.000727
Tim        -0.000279
Ursula      0.000606
Victor     -0.001836
Wendy       0.002251
Xavier     -0.001092
Yvonne      0.002094
Zelda       0.003271
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html